<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#библиотеки" data-toc-modified-id="библиотеки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>библиотеки</a></span></li><li><span><a href="#df" data-toc-modified-id="df-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>df</a></span></li><li><span><a href="#разделение-признаков" data-toc-modified-id="разделение-признаков-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>разделение признаков</a></span></li><li><span><a href="#обучение-модели" data-toc-modified-id="обучение-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>обучение модели</a></span></li><li><span><a href="#сохранение-в-файл" data-toc-modified-id="сохранение-в-файл-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>сохранение в файл</a></span></li></ul></div>

Обучите логистическую регрессию так, чтобы она определяла тональность текста.

Подсчитайте величину TF-IDF для текстов. Лемматизированные тексты твитов для обучения находятся в файле tweets_lemm_train.csv. Целевой признак вы найдёте в столбце positive.


Обученной моделью классификации определите результаты предсказания для тестовой выборки твитов, которая лежит в файле tweets_lemm_test.csv. В этой выборке целевого признака нет. Сохраните предсказания в столбце positive. 


Таблицу с результатом сохраните как csv-файл, но чтобы тренажёр принял файл, не указывайте расширение (например, назовите файл 'predictions')


Значение accuracy вашей модели должно быть не меньше 0.62.

## библиотеки

In [2]:
import pandas as pd

import numpy as np

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score



import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 



import warnings
warnings.filterwarnings('ignore')

D:\Anaconda\envs\praktikum_env\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
D:\Anaconda\envs\praktikum_env\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', 

## df

In [3]:
try:
    df_train = pd.read_csv ('D:/Data for projects/tweets_lemm_train.csv')
    df_test = pd.read_csv ('D:/Data for projects/tweets_lemm_test.csv')

except: #метод try/except применяется для открытия блокнота по адресу Практикума
    df_train = pd.read_csv ('/datasets/tweets_lemm_train.csv')
    df_test = pd.read_csv ('/datasets/tweets_lemm_test.csv')

In [4]:
corpus = df_train['lemm_text'].values.astype('U')
corpus_test = df_test['lemm_text'].values.astype('U')

In [5]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
tf_idf = count_tf_idf.fit_transform(corpus)
tf_idf_test = count_tf_idf.transform(corpus_test)

[nltk_data] Downloading package stopwords to C:\Users\Валерий
[nltk_data]     Бондарцов\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
tf_idf.shape

(5000, 9737)

In [7]:
tf_idf_test.shape

(3000, 9737)

In [8]:
corpus[0]

'хоть я и школотый но поверь у мы то же самый общество профилировать предмет тип '

## разделение признаков

In [9]:
features = tf_idf
features_test = tf_idf_test
target = df_train['positive']

In [10]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=404)

## обучение модели

In [11]:
best_model_log_reg = None
best_result_log_reg = 0
max_iter_ = 0

for _ in range(1,150,10):
    model = LogisticRegression(random_state=404, solver='lbfgs', max_iter=_) 
    model.fit(features_train,target_train) # обучение модели
    predictions = model.predict(features_valid)
    score = accuracy_score(predictions, target_valid) # расчет качества модели
    
    if score > best_result_log_reg:
        best_model_log_reg = model # сохранение наилучшей модели
        best_result_log_reg = score #  сохранение наилучшего значения метрики accuracy
        max_iter_ = _ # сохранение количества иттераций лучшей модели

print(f'accuracy_score наилучшей модели: {round(best_result_log_reg,4)}, количество иттераций: {max_iter_}')

accuracy_score наилучшей модели: 0.6328, количество иттераций: 31


In [12]:
result = best_model_log_reg.predict(features_test)

In [13]:
result = pd.DataFrame(data=result, columns=['positive'])
result

,positive
0,1
1,0
2,0
3,0
4,1
...,...
2995,1
2996,0
2997,0
2998,0


## сохранение в файл

In [14]:
result.to_csv('D:/Data for projects/file1')

In [15]:
np.savetxt(r'D:/Data for projects/np', best_model_log_reg.predict(features_test), fmt='%d')

In [16]:
df_train

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...
...,...,...,...
4995,Вроде дружили 10 лет и в один момент все разру...,0,вроде дружить год и в один момент весь разруши...
4996,"@m_gulko \r\nПоэтому и поздравляю заранее, что...",0,поэтому и поздравлять заранее что не получитьс...
4997,"RT @kubudojede: черт, ну зачем они переделали ...",0,черта ну зачем они переделать
4998,"RT @xidewojopiba: Вроде бы и любим друг друга,...",0,вроде бы и любим друг друг и быть вместе не мо...


In [17]:
df_test

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...
...,...,...
2995,"жуйк, ачивки в контре — зло! мой младший брат ...",жуйк ачивка в контра зло мой младший брат втян...
2996,Не хочу идти на танцы :( http://t.co/5OdPjbYXOC,не хотеть идти на танец
2997,"На улице пусто!ни людей,ни машин,наверно холод...",на улица пусто ни человек ни машина наверно хо...
2998,По-моему мы и ходили за водкой в три часа за э...,по мой мы и ходить за водка в три час за это ч...
